In [1]:

def exception_handler(exc_type, exc_value, exc_traceback):
    # Ne pas intercepter les interruptions clavier
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return
    # Log l'exception avec la trace complète
    print("".join(traceback.format_exception(exc_type, exc_value, exc_traceback)))

In [2]:
from src.generic.observer import HyperliquidObserver
from src.generic.algo import Algo
import logging
import traceback
import sys
from src.data.db.sqlite_data_service import SQLiteDataService

# Configuration du logging pour afficher les traces d'erreur complètes
logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Définir le hook d'exception
sys.excepthook = exception_handler

from src.generic.cctx_api import Dex

dex = Dex(symbol='BTC', marginCoin='USDC')

data_service = SQLiteDataService()
algo = Algo(dex=dex, data_service=data_service, max_leverage=30)
algo.setup_initial_positions()
#algo.recover_previous_state()
hyperliquid_observer = HyperliquidObserver(address='0x765EaafC85566466EF63bc3D3e1f507526b6Cc82', algo=algo)
hyperliquid_observer.start()


2025-06-12 20:23:48,367 - src.generic.algo - INFO - Session ID set to: af442951-c472-4bd0-b3e6-54f9b72fd65a
2025-06-12 20:23:48,367 - src.generic.algo - INFO - Setting up initial positions...
2025-06-12 20:23:48,368 - src.generic.algo - INFO - Setting leverage to 30 for cross margin trading
2025-06-12 20:23:56,151 - src.generic.algo - INFO - Available amount to trade: 592.062219 - Current price: 107962.0 - Single position quantity : 0.000913997855727015
2025-06-12 20:23:56,152 - src.generic.cctx_api - INFO - Buying 0.004569989278635075 at market price : 107962.0
2025-06-12 20:23:56,152 - src.generic.cctx_api - INFO - api - Creating buy market order: 0.004569989278635075 at 107962.0


Initial buy quantity: 0.004569989278635075 - unit : 0.004569989278635075


2025-06-12 20:23:57,869 - src.generic.algo - INFO - 0Creating open long order: 0.000913997855727015 at 107862.0
2025-06-12 20:23:57,870 - src.generic.cctx_api - INFO - api - Creating buy limit order: 0.000913997855727015 at 107862.0
2025-06-12 20:23:59,460 - src.data.db.sqlite_data_service - INFO - Processing new buy position: BTC/USDC:USDC - LONG
2025-06-12 20:23:59,466 - src.generic.algo - INFO - 0 --> Creating close long order: 0.000913997855727015 at 108062.0
2025-06-12 20:23:59,467 - src.generic.cctx_api - INFO - api - Creating sell limit order: 0.000913997855727015 at 108062.0
2025-06-12 20:24:01,418 - src.data.db.sqlite_data_service - INFO - Processing new sell position: BTC/USDC:USDC - SHORT


In [8]:
data_service.get_all_sessions()
data_service.get_observations_by_session('3548695f-c96a-4f89-9f47-7bb62a989d21')

[{'id': 1,
  'event_type': 'position',
  'symbol': 'BTC/USDC:USDC',
  'user_address': 'unknown',
  'session_id': '3548695f-c96a-4f89-9f47-7bb62a989d21',
  'timestamp': '2025-06-10T22:31:07.133986',
  'data': '{"event_type": "position", "symbol": "BTC/USDC:USDC", "user_address": "unknown", "session_id": "3548695f-c96a-4f89-9f47-7bb62a989d21", "timestamp": "2025-06-10T22:31:07.133986", "data": {"side": "LONG", "size": "0.000989375545320504", "entry_price": "109545.0", "unrealized_pnl": null, "realized_pnl": null, "leverage": null, "margin_used": null, "liquidation_price": null}, "source": "hyperliquid_observer", "status": "created"}',
  'oid': None,
  'price': None,
  'status': 'created',
  'source': 'hyperliquid_observer',
  'created_at': '2025-06-10 20:31:07'},
 {'id': 2,
  'event_type': 'position',
  'symbol': 'BTC/USDC:USDC',
  'user_address': 'unknown',
  'session_id': '3548695f-c96a-4f89-9f47-7bb62a989d21',
  'timestamp': '2025-06-10T22:31:08.868412',
  'data': '{"event_type": "pos

In [4]:
hyperliquid_observer.stop()

In [ ]:
import ccxt

dex = ccxt.hyperliquid({
            'walletAddress': '0x765EaafC85566466EF63bc3D3e1f507526b6Cc82',
            "privateKey": "0x208d00493f51713bd0c42979e66180d38ff0e128198a07c3dbd5231a53f44791",
            'options': {'sandbox': True},
        })


In [ ]:
from hyperliquid.utils.constants import MAINNET_API_URL, TESTNET_API_URL
from hyperliquid.info import Info

isProd = False
wallet_address = '0x765EaafC85566466EF63bc3D3e1f507526b6Cc82'
api_url = MAINNET_API_URL if isProd else TESTNET_API_URL

info = Info(base_url=api_url, skip_ws=True)

In [ ]:
from hyperliquid.websocket_manager import WebsocketManager

def handle_order_updates(data):
    print("Mise à jour d'ordre reçue :", data)

ws_manager = WebsocketManager("wss://api.hyperliquid-testnet.xyz/ws")
ws_manager.subscribe(
    {"type": "orderUpdates", "user": "0x765EaafC85566466EF63bc3D3e1f507526b6Cc82"},
    handle_order_updates
)
ws_manager.run()


In [ ]:
import json
from dacite import from_dict
from src.generic.hyperliquid_ws_model import WsMessage, WsOrder

import websocket
import ssl
import certifi
import threading
import time

class HyperliquidWebSocket:

    def __init__(self, url, address: str, observer: HyperliquidObserver):
        self.url = url
        self.address = address
        self.observer = observer
        self.ws = websocket.WebSocketApp(
            url,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close,
            on_open=self.on_open
        )

    def start_watch(self):
        websocket.enableTrace(False)
        #self.ws.run_forever(sslopt={"context": ssl.create_default_context(cafile=certifi.where())})
        self.ws.run_forever(
            sslopt={"cert_reqs": ssl.CERT_REQUIRED, "ca_certs": certifi.where()},
            #ping_interval=10,
            #ping_timeout=5
        )

    def on_message(self, ws, message):
        msg = json.loads(message)
        channel = msg.get("channel")
        if channel == "orderUpdates":
            # retrieves a list of order updates
            order_updates = from_dict(data_class=WsMessage[WsOrder], data=msg)
            self.observer.handle_order_updates(order_updates.data)
        else:
            print("Autre message :", msg)


    def on_error(self, ws, error):
        print("Erreur :", error)

    def on_close(self, ws, close_status_code, close_msg):
        print("WebSocket fermé :", close_status_code, close_msg)

    def on_open(self, ws):
        user_address = self.address
        subscription_message = {
            "method": "subscribe",
            "subscription": {
                "type": "orderUpdates",
                "user": user_address
            }
        }
        ws.send(json.dumps(subscription_message))

        # keep connection alive with ping
        threading.Thread(target=self.run_ping, daemon=True).start()

    def run_ping(self):
        while True:
            time.sleep(10)
            try:
                self.ws.send(json.dumps({"method": "ping"}))
            except Exception as e:
                print("Erreur ping :", e)
                break

#ws = HyperliquidWebSocket("wss://api.hyperliquid-testnet.xyz/ws", "0x765EaafC85566466EF63bc3D3e1f507526b6Cc82")
#ws.start_watch()


In [ ]:
#from src.generic.hyperliquid_ws_model import WsMessage, WsOrder
#
#class HyperliquidObserver:
#    def __init__(self, address: str, algo: Algo):
#        self.address = address
#        self.algo = algo
#        self.hyperliquid_ws = HyperliquidWebSocket(
#            url="wss://api.hyperliquid-testnet.xyz/ws",
#            address=address,
#            observer=self
#        )
#
#    def handle_order_updates(self, orders: [WsOrder]):
#        for order in orders:
#            print(order)
#            try:
#                if order.status == 'deleted':
#                    self.algo.on_deleted_order(order)
#                elif order.status == 'filled':
#                    self.algo.on_executed_order(order)
#                else:
#                    pass
#            except Exception as e:
#                print(f"Error processing order {order.order.id}: {e}")
#
#    def start(self):
#        self.hyperliquid_ws.start_watch()

algo = Algo(symbol='BTC', marginCoin='USDC', max_leverage=20)
hyperliquid_observer = HyperliquidObserver(address='0x765EaafC85566466EF63bc3D3e1f507526b6Cc82', algo=algo)
hyperliquid_observer.start()

In [ ]:
#from hyperliquid.ccxt.base.types import OrderSide
#from typing import Optional, Literal
#
#from src.generic.cctx_api import Dex
#from src.generic.cctx_model import Order
#
#BUY: OrderSide = 'buy'
#SELL: OrderSide = 'sell'
#
#
#from time import time
#
#@dataclass
#class ExecutedOrder:
#    type: Literal['buy', 'sell']
#    price: float
#    timestamp: time
#
#class ExecutedOrdersTracker:
#    last_executed_orders: [ExecutedOrder] = []
#
#    def add_buy(self, price: float):
#        self.last_executed_orders.append(ExecutedOrder(type='buy', price=price, timestamp=time.time()))
#
#    def add_sell(self, price: float):
#        self.last_executed_orders.append(ExecutedOrder(type='sell', price=price, timestamp=time.time()))
#
#class Algo:
#
#    GAPS = [20, 50, 100, 500, 1000, 1500, 2000, 2500, 3000] # gap price to 100 for testing
#    current_gap_idx = 0
#
#    max_leverage = 40
#    max_long_orders = 3
#    # perpFundsPercentageForInitialLong = 10 # initial percentage to open Long position with PERP funds
#
#    previous_orders: [Order] = []
#    executed_orders_tracker = ExecutedOrdersTracker()
#
#    def get_gap(self):
#        return self.GAPS[self.current_gap_idx]
#
#    def __init__(self, symbol: str, marginCoin: str, max_leverage: int = 40):
#        self.symbol = symbol
#        self.marginCoin = marginCoin
#        self.dex = Dex(symbol, marginCoin)
#        self.max_leverage = max_leverage
#
#    # Initialize the algorithm by setting up initial positions
#    def init_all(self):
#        available_amount = self.dex.get_perp_available_balance()
#        current_price = self.dex.get_current_price()
#        initial_position_qty = available_amount / 6 / current_price
#
#        print(f'Total amount: {available_amount}')
#        print(f'Current price: {current_price}')
#        print(f'Quantity: {initial_position_qty}')
#
#        self.dex.set_cross_margin_leverage(self.max_leverage)
#
#        # buy at market price
#        initial_buy_qty_unit = (available_amount / current_price)
#        initial_buy_qty = initial_buy_qty_unit * self.max_leverage
#        print(f"Initial buy quantity: {initial_buy_qty} - unit : {initial_buy_qty_unit}")
#        self.dex.buy_at_market_price(qty=initial_buy_qty, price=current_price)
#
#        #gap = self.get_gap()
#        # self.create_initial_positions(current_price, initial_position_qty, gap)
#
#
#    def on_canceled_order(self, order: WsOrder):
#        print(f"Order deleted: {order.id} - {order.side} at price {order.price}")
#        print(order)
#        self.previous_orders = [o for o in self.previous_orders if o.id != order.id]
#
#
#    def on_executed_order(self, wsOrder: WsOrder):
#        order = wsOrder.order
#        print(f"Order executed: {order.oid} - {order.side} at price {order.limitPx}")
#        self.remove_from_previous_orders(str(order.oid))
#
#        # assume asset price is the limit price of the order because it has just been executed
#        asset_price = order.limitPx
#
#        if order.side == 'B':
#            min_long_price = self.get_min_long_order() - self.get_gap()
#            qty = self.compute_adaptive_order_qty(asset_price)
#            self.executed_orders_tracker.add_buy(order.limitPx)
#            self.dex.buy_at_market_price(qty=qty, price=min_long_price)
#        elif order.side == 'S':
#            new_short_price = order.limitPx + self.get_gap()
#            qty = self.compute_adaptive_order_qty(asset_price) # TODO: check if we must short the adaptive quantity
#            self.executed_orders_tracker.add_sell(order.limitPx)
#            self.dex.open_short_position(qty=qty, price=new_short_price)
#
#
#    def get_min_long_order(self) -> float:
#        # retrives prder with min price from previous orders
#        if not self.previous_orders:
#            return 0.0
#        min_long_order = min(
#            (order for order in self.previous_orders if order.side == BUY),
#            key=lambda o: o.price,
#            default=None
#        )
#        return min_long_order.price if min_long_order else 0.0
#
#    def remove_from_previous_orders(self, order_id: str):
#        self.previous_orders = [o for o in self.previous_orders if o.id != order_id]
#
#    def compute_initial_qty(self, current_price: float) -> float:
#        """Compute the initial quantity based on available funds and current price."""
#        available_funds = self.dex.get_perp_available_balance()
#        return (available_funds / 6) / current_price
#
#
#    def create_initial_positions_from_start(self, current_price: float):
#        order_qty = self.compute_adaptive_order_qty(current_price)
#        self.create_initial_positions(qty=order_qty, current_price=current_price, gap = self.get_gap())
#
#
#    # create initial long and short positions
#    def create_initial_positions(self, qty: float, current_price: float, gap: float):
#        # init short position
#        short_price = current_price + gap
#        order = self.dex.create_close_long(qty, short_price)
#        self.previous_orders.append(order)
#
#        # init long positions
#        for i in range(1, self.max_long_orders):
#            long_price = current_price - (i * gap)
#            order = self.dex.create_open_long(qty, long_price)
#            self.previous_orders.append(order)
#
#
#    def retrieve_previous_orders(self):
#        self.previous_orders = self.dex.get_open_orders()
#
#    # retrieves current orders from dex and checks if they have been executed
#    # if some have been executed creates new ones
#    # def check_orders(self):
#    #     current_open_orders = self.dex.get_open_orders()
#    #     previous_order_ids = [o.id for o in self.previous_orders]
#    #     executed_orders = [order for order in current_open_orders if order.id not in previous_order_ids]
#    #
#    #     lazy_current_price = self.LazyCurrentPrice(self.dex)
#    #
#    #     if len(current_open_orders) == 0:
#    #         # all have been executed
#    #         self.create_initial_positions_from_start(current_price=lazy_current_price.get())
#    #
#    #     # TODO : check if many orders were executed. If true it could be more interesting to do something else
#    #     if executed_orders:
#    #         print(f"Executed order: {executed_orders[0].id} : {executed_orders[0].side} at price {executed_orders[0].price}")
#    #         asset_price = lazy_current_price.get()
#    #         position_qty = self.compute_adaptive_order_qty(asset_price)
#    #
#    #         if self.should_place_short(asset_price, current_open_orders):
#    #             short_price = asset_price + self.get_gap()
#    #             print(f"Creating short limit order: {position_qty} at {short_price}")
#    #             #self.dex.createShortLimit(position_qty, short_price)
#    #
#    #         if self.should_place_long(asset_price, current_open_orders):
#    #             long_price = asset_price - self.get_gap()
#    #             print(f"Creating long limit order: {position_qty} at {long_price}")
#    #             #self.dex.createLongLimit(position_qty, long_price)
#
#
#    def compute_fixed_order_qty(self, asset_price: float) -> float:
#        account_data = self.dex.get_account_data()
#        return account_data.positionMargin / 6 / asset_price
#
#    ## adaptive quantity : because takes in account the margin account
#    def compute_adaptive_order_qty(self, asset_price: float) -> float:
#        account_data = self.dex.get_account_data()
#        return (account_data.totalPositionValue - account_data.positionMargin) / 6 / asset_price
#
#
#    def should_place_short(self, asset_price, orders: [Order]) -> bool:
#        order = self.find_order_in_range(asset_price, orders, SELL, self.get_gap())
#        if order:
#            print(f"Order already exists in range: {order.get('id') - order.get('price')}")
#            return False
#        return True
#
#
#    def should_place_long(self, asset_price: float, orders: [Order]) -> bool:
#        order = self.find_order_in_range(asset_price, orders, BUY, self.get_gap())
#        if order:
#            print(f"Order already exists in range: {order.get('id') - order.get('price')}")
#            return False
#        return True
#
#    def find_order_in_range(self, asset_price, orders: [Order], order_type: OrderSide, gap: float) -> Optional[any]:
#        target_price = asset_price + gap
#        price_range_start = target_price - (gap)
#        price_range_end = target_price - (gap)
#
#        order_in_range = list(filter(lambda o:
#                                    order_type == o.side and
#                                    price_range_start <= o.price <= price_range_end, orders))
#        if len(order_in_range) > 0:
#            return order_in_range[0]
#        return None
#
#
#    class LazyCurrentPrice:
#        current_price: float = None
#
#        def __init__(self, dex: Dex):
#            self.dex = dex
#
#        def get(self) -> float:
#            """Fetch the current price from the dex."""
#            if self.current_price is None:
#                self.current_price = self.dex.get_current_price()
#            return self.current_price

In [ ]:
from src.generic.cctx_api import Dex
dex = Dex(symbol='BTC', marginCoin='USDC')
data = dex.transfer_from_perp_to_spot(10)
print(data)